In [5]:
import requests
import json
from time import sleep
import csv
import urllib
import datetime

# generates a dictionary to be passed in a requests GET method to generate the request header
def generateHeaderDictionary(acceptMediaType):
    requestHeaderDictionary = {
        'Accept' : acceptMediaType
    }
    return requestHeaderDictionary

# write a list of dictionaries to a CSV file
def writeDictsToCsv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csvFileObject:
        writer = csv.DictWriter(csvFileObject, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

endpointUrl = 'https://wiki.bgbm.org/proxy/wdqs/bigdata/namespace/bdi/sparql'

In [9]:
# create a string for the query
query = '''
PREFIX bwd: <http://wiki.bgbm.org/entity/>
PREFIX bwdt: <http://wiki.bgbm.org/prop/direct/>

SELECT ?concept ?conceptLabel ?name ?definition ?usage ?typeLabel  WHERE {
  ?concept bwdt:P8 bwd:Q219.
  ?concept bwdt:P9 ?type.
  ?concept bwdt:P25 ?name.
  ?concept schema:description ?definition.
  ?concept bwdt:P62 ?usage.
#  ?concept bwdt:P6 ?example.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
'''
#print(query)

returnValue = []
acceptMediaType = 'application/json'
r = requests.get(endpointUrl, params={'query' : query}, headers=generateHeaderDictionary(acceptMediaType))
data = r.json()
results = data['results']['bindings']
print(json.dumps(results, indent=2))

[
  {
    "concept": {
      "type": "uri",
      "value": "http://wiki.bgbm.org/entity/Q102"
    },
    "usage": {
      "xml:lang": "en",
      "type": "literal",
      "value": "This element is used to give a broad overview of the content of the data set. More detailed information can be expressed in abcd:TaxonomicTerm and abcd:GeoecologicalTerm."
    },
    "definition": {
      "xml:lang": "en",
      "type": "literal",
      "value": "Free-form text terminology or descriptions available in a data source (geographic, taxonomic, etc.)."
    },
    "name": {
      "type": "literal",
      "value": "coverage"
    },
    "conceptLabel": {
      "xml:lang": "en",
      "type": "literal",
      "value": "Coverage"
    },
    "typeLabel": {
      "xml:lang": "en",
      "type": "literal",
      "value": "Datatype Property"
    }
  },
  {
    "concept": {
      "type": "uri",
      "value": "http://wiki.bgbm.org/entity/Q117"
    },
    "usage": {
      "xml:lang": "en",
      "type": "lit

In [14]:
header = ['term_localName', 'label', 'definition', 'usage', 'rdf_type']
table = []

for result in results:
    resultsDict = {}
    resultsDict['term_localName'] = result['name']['value']
    resultsDict['label'] = result['conceptLabel']['value']
    resultsDict['definition'] = result['definition']['value']
    resultsDict['usage'] = result['usage']['value']
    className = result['typeLabel']['value']
    if className == 'Datatype Property':
        resultsDict['rdf_type'] = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'
    elif className == 'Object Property':
        resultsDict['rdf_type'] = 'http://www.w3.org/1999/02/22-rdf-syntax-ns#Property'
    else:
        resultsDict['rdf_type'] = 'http://www.w3.org/2000/01/rdf-schema#Class'        
    table.append(resultsDict)

filename = 'abcd.csv'
writeDictsToCsv(table, filename, header)

print('Done')

Done
